In [1]:
* Zhengting (Johnathan) He
* May 8th, 2021
* healthy-aging project
* Verify Yaxi's code on generting survival time: 98_14wave.do

In [2]:
// set working directories
global root "F:\Box Sync\Archives2020LLY\Zhengting\Duke Kunshan University Intern (zh133@duke.edu)\4 healthy aging-CLHLS\Group meeting coordination\survival time"
* define path for data sources
global RAW "${root}/raw data"
* define path for output data
global OUT "${root}/out data"
* define path for INTERMEDIATE
global INTER "${root}/inter data"

In [3]:
use "${RAW}/1998_2014_longitudinal_dataset_released_version1.dta", clear

In [4]:
// codebook on death variables

// validated death year
* d0vyear, d2vyear, d5vyear, d8vyear, d11vyear, d14vyear: validated year of death*/
* -9:lost to follow up in the 2000/2002/2005/2008/2011/2014 survey*
* -8:died or lost to follow-up in previous waves*
* -7:it is for the deceased persons, not applicable for survivors*
* 8888: don't know*/
* 9999: missing*/

// validated death month
* d0vmonth, d2vmonth, d5vmonth, d8vmonth, d11vmonth, d14vmonth: validated month of death*/
* -9: lost to follow-up in the 2000/02/05/08/11/14 survey*/
* -8: died or lost to follow-up in previous waves*/
* -7: it is for the deceased persons, not applicable for survivors*/
* 88: don't know*/
* 99: missing*/

// validated death day
* d0vday, d2vday, d5vday, d8vday, d11vday, d14vday: validated day of death*/
* -9: lost to follow-up in the 2000/02/05/08/11/14 survey*/ 
* -8: died or lost to follow-up in previous waves*/
* -7: it is for the deceased persons, not applicable for survivors*/
* 88: don't know*/
* 99: missing*/

* note: d14vyear/month/day & d11vday have . as missing value

// survival status
* dth98_00, dth00_02, dth02_05, dth05_08, dth08_11, dth11_14: status of survival, death, or lost to follow-up from 1998-2000/2000-2002/2002-2005/2005-2008/2008-2011/2011-2014 waves*/
* dth98_00:
* -9:lost to follow-up in the 2000 survey;
* 0: still alive at 2000 survey;
* 1: died before 2000 survey
* dth**_##:
* -9:lost to follow-up at the ## survey;
* -8:died or lost to follow-up in previous waves;
* 0:surviving at the ## survey;
* 1: died before the ## survey

In [5]:
// check whether there are logical mistakes for dth**_##
* If the current death status is -9/0/1, the previous one can only be 0;
* if the current death status is -8, then the previous can only be -8,-9 and 1.
preserve
rename dth98_00 dth1
rename dth00_02 dth2
rename dth02_05 dth3
rename dth05_08 dth4
rename dth08_11 dth5
rename dth11_14 dth6
label drop _all
forv i=1/5 {
    local j=`i'+1
    tab dth`i' if dth`j' == -9 | dth`j' == 0 | dth`j' == 1, missing //0
    tab dth`i' if dth`j' == -8, missing //-8, -9, 1
}
restore











  status of |
  survival, |
  death, or |
    lost to |
  follow-up |
  from 1998 |
    to 2000 |
      waves |      Freq.     Percent        Cum.
------------+-----------------------------------
         -9 |          1        0.02        0.02
          0 |      4,831       99.98      100.00
------------+-----------------------------------
      Total |      4,832      100.00

  status of |
  survival, |
  death, or |
    lost to |
  follow-up |
  from 1998 |
    to 2000 |
      waves |      Freq.     Percent        Cum.
------------+-----------------------------------
         -9 |        893       20.96       20.96
          1 |      3,368       79.04      100.00
------------+-----------------------------------
      Total |      4,261      100.00

  status of |
  survival, |
  death, or |
    lost to |
  follow-up |
  from 2000 |
    to 2002 |
      waves |      Freq.     Percent        Cum.
------------+-----------------------------------
          0 |      2,643      10

In [6]:
* id = 45107898 & 50001898 were problematic
* id=50001898, died in 02_05wave, should change dth98_00/d0vyear/month/day from -9 to 0/-7
* id=45107898, have two died dates, 02_05wave:2002.8.29, and 05_08wave:2006.12.21, both dth02_05 and dth05_08 are 1. Inferring from the data, the person should die in 05_05wave, as there is detailed info in 2005 for that person.

In [7]:
replace dth98_00 = 0 if id == 50001898 
replace dth02_05 = 0 if id == 45107898
replace d5vday = -7 if id == 45107898
replace d5vmonth = -7 if id == 45107898
replace d5vyear = -7 if id == 45107898


(1 real change made)

(1 real change made)

(1 real change made)

(1 real change made)

(1 real change made)


In [8]:
keep id dth98_00 dth00_02 dth02_05 dth05_08 dth08_11 dth11_14 d0vyear d0vmonth d0vday d2vyear d2vmonth d2vday d5vyear d5vmonth d5vday d8vyear d8vmonth d8vday d11vyear d11vmonth d11vday d14vyear d14vmonth d14vday

In [9]:
rename dth98_00 dth0
rename dth00_02 dth2
rename dth02_05 dth5
rename dth05_08 dth8
rename dth08_11 dth11
rename dth11_14 dth14

In [10]:
global waves "0 2 5 8 11 14"
global year1 "1999 2001 2002 2003 2005 2006 2007 2009 2010 2011 2013 2014"
global year2 "2000 2004 2008 2012"
global months "4 6 9 11"
global wavein "in98 in0 in2 in5 in8 in11 in14"
save "${INTER}/work.dta",replace







file F:\Box Sync\Archives2020LLY\Zhengting\Duke Kunshan University Intern (zh133
> @duke.edu)\4 healthy aging-CLHLS\Group meeting coordination\survival time/inte
> r data/work.dta saved


In [11]:
// check whether there are logical mistakes between d*vyear d*vmonth d*vday dth**_##
foreach i of global waves {
    recode d`i'vday(. 88=99) 
    recode d`i'vmonth(. 88=99)
    recode d`i'vyear(. 88 9999=99)  //no 88 for all the 4 vars
    
    replace d`i'vyear = 1 if d`i'vyear > 1997 & d`i'vyear < 2020
    replace d`i'vmonth = 1 if d`i'vmonth > 0 & d`i'vmonth < 13
    replace d`i'vday = 1 if d`i'vday > 0 & d`i'vday < 32
    
    bys d`i'vyear: gen fre`i'_year = _N
    bys d`i'vmonth: gen fre`i'_month = _N
    bys d`i'vday: gen fre`i'_day = _N
    bys dth`i': gen fre`i'_dth = _N
}
label drop _all
save "${INTER}/work1.dta", replace


(d0vday: 0 changes made)
(d0vmonth: 0 changes made)
(d0vyear: 17 changes made)
(3,351 real changes made)
(3,025 real changes made)
(3,181 real changes made)
(d2vday: 0 changes made)
(d2vmonth: 0 changes made)
(d2vyear: 0 changes made)
(1,604 real changes made)
(1,456 real changes made)
(1,517 real changes made)
(d5vday: 0 changes made)
(d5vmonth: 0 changes made)
(d5vyear: 12 changes made)
(1,295 real changes made)
(1,184 real changes made)
(1,173 real changes made)
(d8vday: 0 changes made)
(d8vmonth: 0 changes made)
(d8vyear: 0 changes made)
(480 real changes made)
(446 real changes made)
(467 real changes made)
(d11vday: 9 changes made)
(d11vmonth: 0 changes made)
(d11vyear: 0 changes made)
(177 real changes made)
(161 real changes made)
(164 real changes made)
(d14vday: 9020 changes made)
(d14vmonth: 9018 changes made)
(d14vyear: 9018 changes made)
(75 real changes made)
(63 real changes made)
(71 real changes made)


file F:\Box Sync\Archives2020LLY\Zhengting\Duke Kunshan Universit

In [12]:
foreach i of global waves{
    keep d`i'vyear d`i'vmonth d`i'vday dth`i' fre`i'_year fre`i'_month fre`i'_day fre`i'_dth 
    duplicates drop d`i'vyear d`i'vmonth d`i'vday dth`i', force 
    save "${INTER}/wave`i'.dta", replace
    use "${INTER}/work1.dta", clear
}
use "${INTER}/wave14.dta",clear
append using "${INTER}/wave0.dta" "${INTER}/wave2.dta" "${INTER}/wave5.dta" "${INTER}/wave8.dta" "${INTER}/wave11.dta"



Duplicates in terms of d0vyear d0vmonth d0vday dth0

(9,086 observations deleted)
file F:\Box Sync\Archives2020LLY\Zhengting\Duke Kunshan University Intern (zh133
> @duke.edu)\4 healthy aging-CLHLS\Group meeting coordination\survival time/inte
> r data/wave0.dta saved

Duplicates in terms of d2vyear d2vmonth d2vday dth2

(9,088 observations deleted)
file F:\Box Sync\Archives2020LLY\Zhengting\Duke Kunshan University Intern (zh133
> @duke.edu)\4 healthy aging-CLHLS\Group meeting coordination\survival time/inte
> r data/wave2.dta saved

Duplicates in terms of d5vyear d5vmonth d5vday dth5

(9,083 observations deleted)
file F:\Box Sync\Archives2020LLY\Zhengting\Duke Kunshan University Intern (zh133
> @duke.edu)\4 healthy aging-CLHLS\Group meeting coordination\survival time/inte
> r data/wave5.dta saved

Duplicates in terms of d8vyear d8vmonth d8vday dth8

(9,089 observations deleted)
file F:\Box Sync\Archives2020LLY\Zhengting\Duke Kunshan University Intern (zh133
> @duke.edu)\4 healthy ag

In [25]:
/* The results show that, in wave0-wave11, -9, -8, 0/-7(alive) have completely the same freq, //I am not sure about this statement
all missing values in d*vyear/month/day occur only when dth*=1(died). Only in wave14, all is missing in d14vyear/month/day when dth14=-9/-8/0.
There is no logical mistakes between the 4 vars.*/

In [26]:
// tabulate the lost,died and alive number for each wave
use "${INTER}/work1.dta",clear
foreach i of global waves{
    tabulate dth`i' if dth`i' !=-8
}




  status of |
  survival, |
  death, or |
    lost to |
  follow-up |
  from 1998 |
    to 2000 |
      waves |      Freq.     Percent        Cum.
------------+-----------------------------------
         -9 |        893        9.82        9.82
          0 |      4,832       53.14       62.96
          1 |      3,368       37.04      100.00
------------+-----------------------------------
      Total |      9,093      100.00

  status of |
  survival, |
  death, or |
    lost to |
  follow-up |
  from 2000 |
    to 2002 |
      waves |      Freq.     Percent        Cum.
------------+-----------------------------------
         -9 |        585       12.11       12.11
          0 |      2,643       54.70       66.80
          1 |      1,604       33.20      100.00
------------+-----------------------------------
      Total |      4,832      100.00

  status of |
  survival, |
  death, or |
    lost to |
  follow-up |
   from the |
    2002 to |
 2005 waves |      Freq.     Percent   

In [28]:
foreach i of global waves{
    erase "${INTER}/wave`i'.dta"
}
erase "${INTER}/work1.dta"